# API SNCF: Les retards de train

__Auteur :__ 

Steve Caron

__Présentation :__ 

Ce script permet de requêter les informations sur les arrivées des trains en gare pour la journée d'hier. 

Il retourne les réponses des requêtes dans deux répertoires:

* data/arrivees/ : répertoire contenant les informations sur les arrivées en gare

* data/perturbations/ : répertoire contenant les informations sur les perturabations observées sur les arrivées en gare


__Inputs :__ 

Un fichier data/top{n}gare.json contenant les informations permettant de faire les requêtes pour chaques gares. Les enregistrements sont classés en fonction de la fréquentation des gares.

__Params :__

* CLEF_API : nom sous lequel est enregistrer la cle API

In [16]:
from dotenv import load_dotenv
import os
import requests
import json
import datetime

In [17]:
CLEF_API = "API_KEY"

In [20]:
def convertir_en_string(dt):
    '''Cette fonction convertit un datetime en chaîne de caractères'''
    return datetime.datetime.strftime(dt,'%Y%m%dT%H%M%S')

In [21]:
def convertir_en_datetime(str):
    '''Cette fonction convertit une chaîne de caractères en datetime'''
    return datetime.datetime.strptime((str[:8]+str[9:]),'%Y%m%d%H%M%S')

In [22]:
def to_json(data,nom_fichier):
    '''Cette fonction permet d'enregistrer un fichier JSON'''
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [23]:
def requete_api(code_gare,code_reseau,date):
    '''Cette fonction effectue une requête API pour collecter la liste des arrivées pour une gare spécifique sur un réseau spécifique '''

    base_url = "https://api.sncf.com/v1/coverage/sncf"
    #Requete sans le filtre sur les trains
    requete = f"{base_url}/stop_areas/{code_gare}/networks/{code_reseau}/arrivals?from_datetime={date}"
    reponse = requests.get(requete, auth=(api_key,""))
    reponse_json = reponse.json()
    to_json(reponse_json,"_test.json")
    
    return reponse_json

In [24]:
def requete_entre_dates(code_gare,code_reseau,date_min,date_max,liste_arrivee,liste_perturbation,compteur_requete):
    '''Cette fonction permet de faire des requêtes pour récupérer des données sur tous les enregistrements de la journée.
    Elle sépare en deux listes les informations concernant les départs et les informations concernant les arrivées'''

    derniere_requete = date_min

    print(f"Première requete pour la gare {code_gare} à la date {derniere_requete} sur le reseau {code_reseau}")

    while derniere_requete < date_max:
        # Requete api
        reponse = requete_api(code_gare,code_reseau,derniere_requete)
        compteur_requete += 1
        # extrait les arrivées
        arrivees = reponse["arrivals"]
        # extrait les perturbations
        perturbations = reponse["disruptions"]
        # Ajoute chaque arrivées de la requete à la liste
        [liste_arrivee.append(arrivee) for arrivee in arrivees]
        # Ajoute chaque perturbations de la requete à la liste
        [liste_perturbation.append(perturbation)  for perturbation in perturbations]
        # Si il a moins de 10 arrivées, alors la prochaine requete ne donnera rien, on retourne donc directement les résultats
        if len(arrivees) < 10:
            print(f"Dernière requete pour la gare {code_gare} à la date {derniere_requete} sur le reseau {code_reseau} // dernieres requetes {len(arrivees)} arrivées")
            return liste_arrivee,liste_perturbation,compteur_requete
        
        derniere_requete = arrivees[-1]["stop_date_time"]["base_arrival_date_time"]
    
    print(f"Dernière requete pour la gare {code_gare} à la date {derniere_requete} sur le reseau {code_reseau}")
    
    return liste_arrivee,liste_perturbation,compteur_requete


In [25]:
def liste_id(nom_fichier):
    '''Cette fonction ouvre un fichier json et récupère une liste de toutes les clés d'un dictionnaire'''

    #Ouverture du fichier csv
    with open(nom_fichier,"r") as jsonfil:
        data_gare = json.load(jsonfil)
    toutes_id = data_gare["id"]
    liste_cles_ranked = []
    # J'ajoute toutes les clés du dictionnaire id dans une liste
    [liste_cles_ranked.append(cle) for cle in toutes_id.keys()]
    return data_gare,liste_cles_ranked

In [26]:
def run(data_gare,cle,date_min,date_max):
    '''Cette fontion lance les fonctions pour traiter les données d'une gare'''
    code_gare = data_gare["id"].get(cle)
    nom_gare = data_gare["nom"].get(cle)
    print(f"Debut des requetes pour la gare: {nom_gare}")

    compteur_requete = 0
    liste_code_reseau = []
    liste_arrivees = []
    liste_perturbations = []

    #Récupération des identifiant réseaux pour la gare en cours
    [liste_code_reseau.append(reseau["id"]) for reseau in data_gare["networks"].get(cle)]

    # Je traite réseau par réseau
    for reseau in liste_code_reseau:
        liste_arrivees, liste_perturbations,compteur_requete = requete_entre_dates(code_gare,reseau,date_min,date_max,liste_arrivees,liste_perturbations,compteur_requete)
    
    print(f"Fin des requetes pour la gare :{nom_gare} \n {compteur_requete} requetes effectuées")
    
    nom_fichier_arrivees = f"data/arrivees/{code_gare}-{date_max}.json".replace(":","_")
    nom_fichier_perturbation = f"data/perturbations/{code_gare}-{date_max}.json".replace(":","_")
    to_json(liste_arrivees,nom_fichier_arrivees)
    to_json(liste_perturbations,nom_fichier_perturbation)

In [18]:
# Récupération de la clé API
load_dotenv()
api_key = os.getenv(CLEF_API)

In [19]:
# Creation des string date permettant de faire les requete API 
aujourdhui = datetime.date.today()
hier_debut_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=0, minute=0 ,second=0)
hier_fin_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=23, minute=59 ,second=59)
date_min = convertir_en_string(hier_debut_journee)
date_max = convertir_en_string(hier_fin_journee)

In [28]:
# Récupération des clés dictionnaire contenant les informations des gares
data_gare,liste_cle = liste_id("data/top200gare.json")
# Exécution du run pour chaque gare
for cle in liste_cle[:10]:
    run(data_gare,cle,date_min,date_max)